In [ ]:
!nvidia-smi



Sun Oct  5 12:40:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
pip -q install git+https://github.com/andreinechaev/nvcc4jupyter


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter --force-reinstall


  Cloning https://github.com/andreinechaev/nvcc4jupyter to /tmp/pip-req-build-507s3kdy
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter /tmp/pip-req-build-507s3kdy
  Resolved https://github.com/andreinechaev/nvcc4jupyter to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-kxc1mne4/wheels/1e/39/04/2b22de7f214011eda71e8d3ea299fa90c5abf31d866ddf7847
Successfully built nvcc4jupyter
  Attempting uninstall: nvcc4jupyter
    Found existing installation: nvcc4jupyter 1.2.1
    Uninstalling nvcc4jupyter-1.2.1:
      Successfully uninstalled nvcc4jupyter-1.2.1


In [ ]:
!nvcc --version



nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter


  Cloning https://github.com/andreinechaev/nvcc4jupyter to /tmp/pip-req-build-5qnyqpj0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter /tmp/pip-req-build-5qnyqpj0
  Resolved https://github.com/andreinechaev/nvcc4jupyter to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!nvidia-smi --query-gpu=name --format=csv,noheader
!nvcc --version


Tesla T4
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [1]:
%%writefile cuda_matmul.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

#define CUDA_CHECK(call)                                            \
    do {                                                            \
        cudaError_t err = call;                                     \
        if (err != cudaSuccess) {                                   \
            fprintf(stderr, "CUDA error %s:%d: %s\n",               \
                    __FILE__, __LINE__, cudaGetErrorString(err));   \
            exit(1);                                                \
        }                                                           \
    } while (0)

static void fill_test(float *A, float *B, int N) {
    // A: pattern 1..7, B: identity
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            A[(size_t)i * N + j] = (float)((i + j) % 7 + 1);
            B[(size_t)i * N + j] = (i == j) ? 1.0f : 0.0f;
        }
    }
}

static void print_mat(const char *name, const float *M, int N) {
    printf("%s:\n", name);
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j)
            printf("%6.1f ", M[(size_t)i * N + j]);
        printf("\n");
    }
}

// Each thread computes one C[row, col]
__global__ void matmul_kernel(const float *A, const float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row >= N || col >= N) return;

    float sum = 0.0f;
    for (int k = 0; k < N; ++k) {
        sum += A[(size_t)row * N + k] * B[(size_t)k * N + col];
    }
    C[(size_t)row * N + col] = sum;
}

int main(int argc, char **argv) {
    if (argc < 3) {
        fprintf(stderr, "Usage: %s N blockDimX [blockDimY] [--print]\n", argv[0]);
        return 1;
    }

    const int N          = atoi(argv[1]);
    const int blockDimX  = atoi(argv[2]);
    const int blockDimY  = (argc >= 4 && argv[3][0] != '-') ? atoi(argv[3]) : blockDimX;
    const bool do_print  =
        (argc >= 4 && strcmp(argv[argc-1], "--print") == 0);

    if (N <= 0 || blockDimX <= 0 || blockDimY <= 0) {
        fprintf(stderr, "N, blockDimX, blockDimY must be positive.\n");
        return 1;
    }

    if (blockDimX * blockDimY > 1024) {
        fprintf(stderr, "Error: blockDimX * blockDimY must be <= 1024 (hardware limit).\n");
        return 1;
    }

    const size_t bytes = (size_t)N * (size_t)N * sizeof(float);

    // Host memory
    float *hA = (float*)malloc(bytes);
    float *hB = (float*)malloc(bytes);
    float *hC = (float*)malloc(bytes);
    if (!hA || !hB || !hC) {
        fprintf(stderr, "Host allocation failed.\n");
        return 1;
    }

    fill_test(hA, hB, N);

    // Device memory
    float *dA = nullptr, *dB = nullptr, *dC = nullptr;
    CUDA_CHECK(cudaMalloc(&dA, bytes));
    CUDA_CHECK(cudaMalloc(&dB, bytes));
    CUDA_CHECK(cudaMalloc(&dC, bytes));

    // Copy inputs to device
    CUDA_CHECK(cudaMemcpy(dA, hA, bytes, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB, bytes, cudaMemcpyHostToDevice));

    // Configure grid and block
    dim3 block(blockDimX, blockDimY);
    dim3 grid((N + block.x - 1) / block.x,
              (N + block.y - 1) / block.y);

    // Timing with CUDA events
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    CUDA_CHECK(cudaEventRecord(start));
    matmul_kernel<<<grid, block>>>(dA, dB, dC, N);
    CUDA_CHECK(cudaEventRecord(stop));

    CUDA_CHECK(cudaEventSynchronize(stop));
    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));

    // Copy result back
    CUDA_CHECK(cudaMemcpy(hC, dC, bytes, cudaMemcpyDeviceToHost));

    // ms -> seconds
    double time_s = ms / 1000.0;

    printf("N=%d, blockDim=(%d,%d), grid=(%d,%d), time=%.6f S \n", N, blockDimX, blockDimY, grid.x, grid.y, time_s);

    if (do_print && N <= 8) {
        print_mat("A", hA, N);
        print_mat("B", hB, N);
        print_mat("C = A*B", hC, N);
    }

    // Cleanup
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dC));
    free(hA); free(hB); free(hC);

    return 0;
}

Writing cuda_matmul.cu


In [2]:
!nvcc -O3 cuda_matmul.cu -o cuda_matmul

In [25]:
!./cuda_matmul 2048 8 8

N=2048, blockDim=(8,8), grid=(256,256), time=0.011348 S 


In [27]:
!./cuda_matmul 2048 16 16

N=2048, blockDim=(16,16), grid=(128,128), time=0.011280 S 


In [37]:
!./cuda_matmul 2048 32 16

N=2048, blockDim=(32,16), grid=(64,128), time=0.007760 S 


In [58]:
!./cuda_matmul 2048 32 32

N=2048, blockDim=(32,32), grid=(64,64), time=0.007393 S 
